In [1]:
import pandas as pd
import numpy as np

In [2]:
from dynamic_portfolio.preprocess import ready_to_train_df
from dynamic_portfolio.utils import *

In [5]:
clean_data(features_creation('OTIS'))

/home/racho961/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/home/racho961/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,...,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,5406460.0,0.00,1.0,0.47,0.0,0.0063,0.0020,0.0043,-36.98,157005.0,...,0.005186,-0.030769,0.000000,-0.044444,-3.019661,0.001804,0.0,-0.002276,-0.147815,-0.087499
1,6845537.0,0.00,1.0,0.47,0.0,0.0058,0.0020,0.0038,8.91,157005.0,...,-0.004254,-0.079365,0.000000,-0.116279,-1.240941,0.003001,0.0,-0.002276,-0.147815,-0.087499
2,6801923.0,0.00,1.0,0.47,0.0,0.0063,0.0022,0.0041,13.64,157005.0,...,0.016449,0.086207,0.100000,0.078947,0.530864,0.001297,0.0,-0.002276,-0.147815,-0.087499
3,5174839.0,0.00,1.0,0.47,0.0,0.0061,0.0022,0.0039,15.06,157005.0,...,0.010434,-0.031746,0.000000,-0.048780,0.104106,0.000398,0.0,-0.002276,-0.147815,-0.087499
4,6446904.0,0.00,1.0,0.47,0.0,0.0060,0.0022,0.0038,15.99,157005.0,...,-0.002253,-0.016393,0.000000,-0.025641,0.061753,-0.000498,0.0,-0.002276,-0.147815,-0.087499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2169657.0,0.00,1.0,0.80,0.0,0.0367,0.0435,-0.0068,85.62,273451.0,...,-0.002519,-0.034211,-0.004577,0.192982,-0.014389,-0.001128,0.0,0.000000,0.000000,-0.001197
643,3420825.0,0.29,1.0,0.80,0.0,0.0377,0.0443,-0.0066,81.69,273451.0,...,-0.007273,0.027248,0.018391,-0.029412,-0.045900,0.003858,0.0,0.000000,0.000000,-0.001197
644,3697558.0,0.00,1.0,0.80,0.0,0.0382,0.0451,-0.0069,80.07,273451.0,...,-0.006321,0.013263,0.018059,0.045455,-0.019831,0.002249,0.0,0.000000,0.000000,-0.001197
645,2068824.0,0.00,1.0,0.80,0.0,0.0383,0.0448,-0.0065,79.74,273451.0,...,-0.006819,0.002618,-0.006652,-0.057971,-0.004121,0.008510,0.0,0.000000,0.000000,-0.001197
